In [8]:
import os
import pandas as pd
import json
import re

PL_WDC_NORMALIZED_PATH = os.path.join(
    os.path.abspath(''),
    '../pl_wdc_normalized.json'
)

products = []
for line in open(PL_WDC_NORMALIZED_PATH, 'r'):
    products.append(json.loads(line))

In [28]:
import re

ner_data_sentences = []
ner_data_tags = []
i_count = 0


for product in products:
    title = product['title']
    title_split = product['title'].split()
    brand_split = product['brand'].split()
    n_brand = len(brand_split)
    n_title = len(title_split)

    product_ner_data_tags = ['O'] * n_title

    p = re.compile(r'\d+ [a-z]+')
    for m in p.finditer(title):
        first_position = title[:m.start()].count(' ')
        product_ner_data_tags[first_position] = 'B-ORG'
        product_ner_data_tags[first_position+1] = 'I-ORG'

    pos = title.find(product['brand'])
    if pos > -1:
        first_position = title[:pos].count(' ')
        for i_brand in range(n_brand):
            tag = 'B-PER'
            if i_brand > 0:
                tag = 'I-PER'
                i_count += 1
            product_ner_data_tags[first_position+i_brand] = tag

    ner_data_sentences.append(title_split)
    ner_data_tags.append(product_ner_data_tags)

print(f"i_count = {i_count}")



for sentence, tags in zip(ner_data_sentences, ner_data_tags):
    print(sentence)
    print(tags)
    print("# # # # #")

i_count = 380
['mola', 'ultra', 'chłonne', 'ręczniki', 'papierowe', '4', 'rolki']
['B-PER', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG']
# # # # #
['woolite', 'delicate', 'płyn', 'do', 'prania', 'do', 'delikatnych', 'tkanin', 'i', 'wełny']
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# # # # #
['air', 'wick', 'life', 'scents', 'pachnące', 'patyczki', 'o', 'zapachu', 'tajemniczych', 'ogrodów', 'z', 'dzieciństwa', '30', 'ml']
['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG']
# # # # #
['tarczyn', 'sok', '100', 'pomarańcza']
['B-PER', 'O', 'B-ORG', 'I-ORG']
# # # # #
['kubuś', 'play', 'napój', 'owocowy', 'woda', 'i', 'owoce', 'jabłko', 'cytryna']
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
# # # # #
['ludwik', 'care', 'płyn', 'do', 'mycia', 'paneli', '1', 'l']
['B-PER', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG']
# # # # #
['ręcznik', 'kuchenny', '2', 'warstwowy', 'celulozowy', '50', 'listków']
['O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG'

In [30]:
output_count = len(ner_data_sentences)

RESULTS_PATH = os.path.join(os.path.abspath(''), "../ner_dataset")

parts = [
    {"label": "train", "start": 0.0, "end": 0.8},
    {"label": "test", "start": 0.8, "end": 1.0},
    {"label": "valid", "start": 0.8, "end": 1.0}
]

for part in parts:
    part_path = os.path.join(RESULTS_PATH, f"{part['label']}.txt")
    start = int(output_count * part['start'])
    end = int(output_count * part['end'])

    result_file = open(part_path, 'w+')

    for idx in range(start, end):
        data_sentence = ner_data_sentences[idx]
        data_tags = ner_data_tags[idx]
        for token, tag in zip(data_sentence, data_tags):
            result_file.write(f'{token}\t{tag}\n')
        result_file.write('\n')
    result_file.close()